In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

import pandas as pd
from scipy import stats

spark = SparkSession.builder \
    .appName("example") \
    .master(f"spark://172.18.0.2:7077") \
    .config("spark.executor.cores", "1") \
    .config("spark.task.cpus", 1) \
    .config("spark.pyspark.virtualenv.enabled", "true") \
    .config("spark.pyspark.virtualenv.type", "native") \
    .config("spark.pyspark.virtualenv.requirements", "/work/reqs/reqs.txt") \
    .getOrCreate()

In [2]:
%%time

big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val',f.rand())
)

big_df.cache()
big_df.show(3)

@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

(
    big_df
    .withColumn('probability', pandas_pdf(big_df.val))
    .show(5)
)

+---+------------------+
| id|               val|
+---+------------------+
|  0|0.6242664344823765|
|  1|0.8289984396250024|
|  2|0.8139291709017323|
+---+------------------+
only showing top 3 rows



/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---+------------------+-------------------+
| id|               val|        probability|
+---+------------------+-------------------+
|  0|0.6242664344823765|0.32831136944229555|
|  1|0.8289984396250024| 0.2829294403942166|
|  2|0.8139291709017323| 0.2864535512287373|
|  3|0.0896133138099815| 0.3973436298889499|
|  4|0.7524495099003106|0.30058380891284725|
+---+------------------+-------------------+
only showing top 5 rows

CPU times: user 203 ms, sys: 11.1 ms, total: 214 ms
Wall time: 11.7 s


In [4]:
pd.Series(stats.norm.pdf(0.5))

0    0.352065
dtype: float64

In [5]:
stats.norm.pdf(0.5)

0.3520653267642995